# Importing packages

In [ ]:
# Basic packages
import pandas as pd
import numpy as np
import re
import random
import collections
import matplotlib.pyplot as plt
from pathlib import Path

# Packages for data preparation
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.optimizers.experimental import RMSprop
import nltk

# Packages for modeling
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import cv2
import os
from PIL import Image

# Load the Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Cataract'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/drive/MyDrive/Cataract/.DS_Store
/content/drive/MyDrive/Cataract/test/.DS_Store
/content/drive/MyDrive/Cataract/test/cataract/image_254.png
/content/drive/MyDrive/Cataract/test/cataract/image_252.png
/content/drive/MyDrive/Cataract/test/cataract/image_251.png
/content/drive/MyDrive/Cataract/test/cataract/image_253.png
/content/drive/MyDrive/Cataract/test/cataract/image_248.png
/content/drive/MyDrive/Cataract/test/cataract/image_249.png
/content/drive/MyDrive/Cataract/test/cataract/image_250.png
/content/drive/MyDrive/Cataract/test/cataract/image_246.png
/content/drive/MyDrive/Cataract/test/cataract/image_256.png
/content/drive/MyDrive/Cataract/test/cataract/image_255.png
/content/drive/MyDrive/Cataract/test/cataract/image_247.png
/content/drive/MyDrive/Cataract/test/cataract/image_277.png
/content/drive/MyDrive/Cataract/test/cataract/image_275.png
/content/drive/MyDrive/Cataract/test/cataract/image_258.png
/content/drive/MyDrive/Cataract/test/cataract/image_264.png
/content/dr

In [ ]:
train_dir = '/content/drive/MyDrive/Cataract/train'
test_dir = '/content/drive/MyDrive/Cataract/test'

In [ ]:
img_height, img_width = 32, 32
batch_size = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size = (img_height, img_width),
    batch_size = batch_size
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 491 files belonging to 2 classes.
Found 121 files belonging to 2 classes.


# Adam Optimizer + Cross Entropy Loss


In [ ]:
model = tf.keras.Sequential(
    [
    #  tf.keras.layers.Rescaling(1./255),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.Dense(2)
    ]
)


In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    ignore_class=None,
    name='sparse_categorical_crossentropy'
),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    validation_data = test_ds,
    epochs = 10
)

Epoch 1/10
31/31 [==============================] - 90s 2s/step - loss: 2.7996 - accuracy: 0.6008 - val_loss: 0.7031 - val_accuracy: 0.6777
Epoch 2/10
31/31 [==============================] - 16s 398ms/step - loss: 0.5232 - accuracy: 0.7821 - val_loss: 0.4616 - val_accuracy: 0.7851
Epoch 3/10
31/31 [==============================] - 16s 397ms/step - loss: 0.3171 - accuracy: 0.8676 - val_loss: 0.3136 - val_accuracy: 0.8926
Epoch 4/10
31/31 [==============================] - 16s 405ms/step - loss: 0.2084 - accuracy: 0.9369 - val_loss: 0.2342 - val_accuracy: 0.9174
Epoch 5/10
31/31 [==============================] - 21s 566ms/step - loss: 0.1401 - accuracy: 0.9613 - val_loss: 0.2457 - val_accuracy: 0.9008
Epoch 6/10
31/31 [==============================] - 16s 366ms/step - loss: 0.1007 - accuracy: 0.9674 - val_loss: 0.2364 - val_accuracy: 0.9174
Epoch 7/10
31/31 [==============================] - 16s 414ms/step - loss: 0.0950 - accuracy: 0.9695 - val_loss: 0.2452 - val_accuracy: 0.8926
Ep

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 32)         0

# Adam + Binary cross

In [ ]:
image_size = (55, 94, 3)
target_size = (55, 94)

train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = target_size,
    class_mode = 'binary'
)

Found 491 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu',input_shape=image_size),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [16]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [17]:
model.fit_generator(
    train_generator,
    epochs=15
)

<ipython-input-17-bd96f6f1d896>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/15
16/16 [==============================] - 16s 939ms/step - loss: 0.1775 - accuracy: 0.9430
Epoch 2/15
16/16 [==============================] - 16s 939ms/step - loss: 0.1293 - accuracy: 0.9470
Epoch 3/15
16/16 [==============================] - 15s 970ms/step - loss: 0.1220 - accuracy: 0.9470
Epoch 4/15
16/16 [==============================] - 15s 944ms/step - loss: 0.1446 - accuracy: 0.9491
Epoch 5/15
16/16 [==============================] - 15s 955ms/step - loss: 0.0837 - accuracy: 0.9633
Epoch 6/15
16/16 [==============================] - 16s 1s/step - loss: 0.0790 - accuracy: 0.9695
Epoch 7/15
16/16 [==============================] - 15s 983ms/step - loss: 0.0466 - accuracy: 0.9857
Epoch 8/15
16/16 [==============================] - 16s 995ms/step - loss: 0.0984 - accuracy: 0.9593
Epoch 9/15
16/16 [==============================] - 15s 963ms/step - loss: 0.1218 - accuracy: 0.9511
Epoch 10/15
16/16 [==============================] - 15s 891ms/step - loss: 0.0406 - accuracy:

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 53, 92, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 46, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 44, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 22, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 8448)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              